In [26]:
import os

base_folder = r"D:\hackethon\codecrafter 2.0\Real-time-intrusion-Classification-and-Alerting\aiml\Voice_detect\data\dataset\Animal-Soundprepros"
if os.path.exists(base_folder):
    print("Folders inside:", os.listdir(base_folder))
else:
    print("❌ Folder does not exist:", base_folder)


Folders inside: ['Aslan', 'Bear', 'Cat', 'Chicken', 'Cow', 'Dog', 'Dolphin', 'Donkey', 'Elephant', 'Frog', 'Horse', 'Monkey', 'Sheep']


In [27]:
# Debug: Print unique folder names
print("📂 Available Folders:", available_folders)

# Debug: Print first 10 names to check matching logic
print("🎵 First 10 file names from CSV:", df["name"].head(10).tolist())


📂 Available Folders: {'Cat', 'Chicken', 'Dolphin', 'Elephant', 'Bear', 'Frog', 'Cow', 'Dog', 'Horse', 'Donkey', 'Aslan', 'Monkey', 'Sheep'}
🎵 First 10 file names from CSV: ['Lion_1.wav', 'Lion_10.wav', 'Lion_11.wav', 'Lion_12.wav', 'Lion_13.wav', 'Lion_14.wav', 'Lion_15.wav', 'Lion_16.wav', 'Lion_17.wav', 'Lion_18.wav']


In [50]:
import os
import pandas as pd

# Load CSV
csv_path = r"D:\hackethon\codecrafter 2.0\Real-time-intrusion-Classification-and-Alerting\aiml\Voice_detect\data\dataset\Animal_Sound.csv"
df = pd.read_csv(csv_path)
print(df.head())

# Define the base audio directory
base_folder = r"D:\hackethon\codecrafter 2.0\Real-time-intrusion-Classification-and-Alerting\aiml\Voice_detect\data\dataset\Animal-Soundprepros"

# 🔹 Get the available folders (animal categories)
available_folders = set(os.listdir(base_folder))  # Set of valid folder names

# 🔹 Extract the correct category (folder name) from "name" column (e.g., "Dog_1.wav" → "Dog")
df["category"] = df["name"].apply(lambda x: x.split("_")[0] if x.split("_")[0] in available_folders else None)

# 🔹 Construct full path
df["full_path"] = df.apply(lambda row: os.path.join(base_folder, row["category"], row["name"]) if row["category"] else None, axis=1)

# 🔹 Check if files exist
df["file_exists"] = df["full_path"].apply(lambda x: os.path.exists(x) if x else False)

# 🔹 Print only "Dog" related rows for verification
df_dog = df[df["category"] == "Dog"]
print(df_dog[["name", "category", "full_path", "file_exists"]].head(10))


          name                                        path  channels  \
0   Lion_1.wav  C:/Users/Admin/Desktop/Animal-Soundprepros         2   
1  Lion_10.wav  C:/Users/Admin/Desktop/Animal-Soundprepros         2   
2  Lion_11.wav  C:/Users/Admin/Desktop/Animal-Soundprepros         1   
3  Lion_12.wav  C:/Users/Admin/Desktop/Animal-Soundprepros         1   
4  Lion_13.wav  C:/Users/Admin/Desktop/Animal-Soundprepros         1   

   sample_width  frame_rate  nframes  duration  size  
0             2       22050    99296  4.503220  4096  
1             2       11025    75964  6.890159  4096  
2             1       22000    31744  1.442909  4096  
3             2       11025    87108  7.900952  4096  
4             1       11025    93972  8.523537  4096  
           name category                                          full_path  \
250   Dog_1.wav      Dog  D:\hackethon\codecrafter 2.0\Real-time-intrusi...   
251  Dog_10.wav      Dog  D:\hackethon\codecrafter 2.0\Real-time-intrusi...   


In [29]:
print(df.groupby("category")["file_exists"].value_counts())


category  file_exists
Bear      True           50
Cat       True           50
Chicken   True           50
Cow       True           50
Dog       True           50
Dolphin   True           50
Donkey    True           50
Elephant  True           50
Frog      True           50
Horse     True           50
Monkey    True           50
Sheep     True           50
Name: count, dtype: int64


In [45]:
import os

save_dir = r"D:\hackethon\codecrafter 2.0\Real-time-intrusion-Classification-and-Alerting\aiml\Voice_detect\data\processed"

# ✅ Create directory if it doesn’t exist
os.makedirs(save_dir, exist_ok=True)


In [46]:
import librosa
import numpy as np
import pandas as pd
import os

# Function to extract features
def extract_features(path):
    try:
        y, sr = librosa.load(path, sr=None)  # Load audio
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Extract MFCCs
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)  # Extract Chroma
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)  # Extract Spectral Centroid
        zcr = librosa.feature.zero_crossing_rate(y)  # Extract Zero-Crossing Rate

        # Take the mean of each feature
        features = np.hstack([
            np.mean(mfccs, axis=1), 
            np.mean(chroma, axis=1),
            np.mean(spectral_centroid),
            np.mean(zcr)
        ])
        return features
    except Exception as e:
        print(f"❌ Error processing {path}: {e}")
        return None

# Read CSV
csv_path = r"D:\hackethon\codecrafter 2.0\Real-time-intrusion-Classification-and-Alerting\aiml\Voice_detect\data\dataset\Animal_Sound.csv"
df = pd.read_csv(csv_path)

# 🔹 **Filter Only 'Dog' Sounds**
df = df[df["path"].str.contains("Dog", case=False, na=False)]

# Extract features for each file
features_list = []
for index, row in df.iterrows():
    file_path = row["path"]

    if os.path.exists(file_path):
        features = extract_features(file_path)
        if features is not None:
            features_list.append([row["name"]] + features.tolist())
    else:
        print(f"⚠️ File Not Found: {file_path}")

# Convert to DataFrame
columns = ["filename"] + [f"mfcc_{i}" for i in range(13)] + [f"chroma_{i}" for i in range(12)] + ["spectral_centroid", "zero_crossing_rate"]
features_df = pd.DataFrame(features_list, columns=columns)

# Save extracted features to CSV
save_path = r"D:\hackethon\codecrafter 2.0\Real-time-intrusion-Classification-and-Alerting\aiml\Voice_detect\data\processed\dog_audio_features.csv"
features_df.to_csv(save_path, index=False)

print("✅ Feature extraction for 'Dog' sounds completed and saved!")


✅ Feature extraction for 'Dog' sounds completed and saved!


In [49]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# Load original CSV
csv_path = r"D:\hackethon\codecrafter 2.0\Real-time-intrusion-Classification-and-Alerting\aiml\Voice_detect\data\dataset\Animal_Sound.csv"
df = pd.read_csv(csv_path)

# Load extracted features CSV
features_path = r"D:\hackethon\codecrafter 2.0\Real-time-intrusion-Classification-and-Alerting\aiml\Voice_detect\data\processed\dog_audio_features.csv"
features_df = pd.read_csv(features_path)

# 🔹 Ensure column names match before merging
features_df.rename(columns={"filename": "name"}, inplace=True)

# 🔹 Standardize 'name' column (lowercase, strip spaces, remove extensions)
df["name"] = df["name"].str.strip().str.lower().str.replace(r"\.\w+$", "", regex=True)
features_df["name"] = features_df["name"].str.strip().str.lower()

# 🔹 Debug: Print unique values
print("Unique values in Original Dataset (name):", df["name"].unique())
print("Unique values in Extracted Features (name):", features_df["name"].unique())

# 🔹 Merge again
df = df.merge(features_df, on="name", how="inner")

# 🔹 Debugging: Check merge result
print("Merged DataFrame shape:", df.shape)
print("Merged DataFrame columns:", df.columns)

# 🔹 Encode labels (if needed)
if not df.empty:
    le = LabelEncoder()
    df["label"] = le.fit_transform(df["name"])

    # 🔹 Convert features into NumPy arrays
    X = df.iloc[:, 2:-1].to_numpy()  # Excluding "name" and "label"
    y = df["label"].to_numpy()

    # 🔹 Split dataset into training/testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("✅ Data preprocessing completed!")
    print("Training data size:", X_train.shape)
    print("Testing data size:", X_test.shape)
else:
    print("❌ Still no data available. Check filename mismatches.")


Unique values in Original Dataset (name): ['lion_1' 'lion_10' 'lion_11' 'lion_12' 'lion_13' 'lion_14' 'lion_15'
 'lion_16' 'lion_17' 'lion_18' 'lion_19' 'lion_2' 'lion_20' 'lion_21'
 'lion_22' 'lion_23' 'lion_24' 'lion_25' 'lion_26' 'lion_27' 'lion_28'
 'lion_29' 'lion_3' 'lion_30' 'lion_31' 'lion_32' 'lion_33' 'lion_34'
 'lion_35' 'lion_36' 'lion_37' 'lion_38' 'lion_39' 'lion_4' 'lion_40'
 'lion_41' 'lion_42' 'lion_43' 'lion_44' 'lion_45' 'lion_46' 'lion_47'
 'lion_48' 'lion_49' 'lion_5' 'lion_50' 'lion_6' 'lion_7' 'lion_8'
 'lion_9' 'bear_1' 'bear_10' 'bear_11' 'bear_12' 'bear_13' 'bear_14'
 'bear_15' 'bear_16' 'bear_17' 'bear_18' 'bear_19' 'bear_2' 'bear_20'
 'bear_21' 'bear_22' 'bear_23' 'bear_24' 'bear_25' 'bear_26' 'bear_27'
 'bear_28' 'bear_29' 'bear_3' 'bear_30' 'bear_31' 'bear_32' 'bear_33'
 'bear_34' 'bear_35' 'bear_36' 'bear_37' 'bear_38' 'bear_39' 'bear_4'
 'bear_40' 'bear_41' 'bear_42' 'bear_43' 'bear_44' 'bear_45' 'bear_46'
 'bear_47' 'bear_48' 'bear_49' 'bear_5' 'bear_50